# RR Lyrae Period-Luminosity

Eric Bellm
January 2022

In this exercise we'll use real data from a variety of surveys to learn more about the relationship between an RR Lyrae variable's period and its luminosity.

In [29]:
%matplotlib notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astropy.table import Table
from astropy import units as u
from astropy.time import Time
from astropy.timeseries import TimeSeries, LombScargle

## 1. Finding the Period of an RR Lyrae star

We begin by loading the ZTF lightcurve of a previously-identified RR Lyrae star.  We've pre-written this function for you.

In [26]:
def load_ztf_lc(filename):
    """Load a ZTF parquet lightcurve saved by `ztf_query`.
    
    See `ZTF_RRLyrae_data.ipynb` for data retrieval.
    
    Parameters
    ----------
    filename : string
        path to a parquet file saved by `ztf_query`
        
    Returns
    -------
    tbl : `astropy.timeseries.TimeSeries`
        lightcurve 
    mean_ra : `float`
        mean right ascension of the source
    mean_dec : `float`
        mean declination of the source
    """
    
#     tbl = Table.read(filename, format='parquet')
    tbl = pd.read_parquet(filename)
    
    # exclude flagged (bad) data
    tbl = tbl[tbl['catflags'] == 0]
    
    # compute average position for this source
    mean_ra = np.mean(tbl['ra'])
    mean_dec = np.mean(tbl['dec'])
    
    return TimeSeries(time=Time(tbl['hjd'].values,format='jd'),
                      data=Table.from_pandas(tbl[['mag','magerr','filtercode']])), mean_ra, mean_dec

We load it into an astropy [`Timeseries`](https://docs.astropy.org/en/stable/timeseries/index.html).

In [ ]:
ts, mean_ra, mean_dec = load_ztf_lc('ztf_rrl_lightcurves/12.parquet')
ts

In [135]:
print(mean_ra, mean_dec)

245.55715820833677 44.55899377304023


Which filters are present?

In [30]:
set(ts['filtercode'])
# this means {ZTF g-band, ZTF r-band, ZTF i-band}

{'zg', 'zi', 'zr'}

Let's assign default colors to these filters... you can change them if you like!

In [31]:
colors = {'zg':'C0',
          'zr':'C1',
          'zi':'C3'}

Next, let's plot the lightcurve.

In [32]:
def plot_lc(ts):
    """Plot a ZTF lightcurve.
    
    Parameters
    ----------
    ts :  `astropy.timeseries.TimeSeries`
        ZTF lightcurve
        
    """
    
    fig = plt.figure() 
    
    # loop over the available filters
    for filt in set(ts['filtercode']):
        # select the rows that correspond to that filter
        wfilt = ts['filtercode'] == filt
        plt.errorbar(ts[wfilt].time.mjd,ts[wfilt]['mag'],ts[wfilt]['magerr'],
                     label=filt, fmt='.', ls='none', color=colors[filt], alpha=0.6)
        
    plt.legend()
    plt.xlabel('MJD (days)')
    plt.ylabel('Magnitude')
    plt.gca().invert_yaxis() # smaller magnitudes are brighter--flip the y axis!

In [ ]:
plot_lc(ts)

Astropy's period finding expects the inputs to be in a single filter.  

Let's define a selector that identifies only the rows with r-band data:

In [140]:
wr = ts['filtercode'] == 'zr'

Here's an example of how to apply it:

In [ ]:
ts[wr]['filtercode']

### Exercise 1 Tasks:

* Compute the Lomb-Scargle periodogram for the r-band lightcurve using the astropy [`LombScargle`](https://docs.astropy.org/en/stable/timeseries/lombscargle.html) class.
* Plot the resulting periodogram (frequency, power) or (period, power).  Label your axes.
* Identify the "best" period from the periodogram (highest peak)
* Plot the phase-folded lightcurve (in all 3 filters) for this period.

## 2. Plot Period-Luminosity relation

If we computed a large number of periods for RR Lyrae for which we knew their distances, we could determine if there is a relationship between period and luminosity, and use it for new stars we discover.  

Computing so many periods is a lot of work, so let's use a recently-published catalog from [Huang and Koposov 2022](https://ui.adsabs.harvard.edu/abs/2022MNRAS.510.3575H/abstract).  We will read in their catalog directly from one of the provided files:

In [38]:
# read using astropy Tables

# tbl = Table.read('rrl_main_cat.csv.gz')
tbl = pd.read_csv('rrl_main_cat.csv.gz')

In [39]:
tbl.columns

Index(['objid', 'source_id', 'ra', 'dec', 'prob_rrl', 'best_period', 'ebv',
       'distance', 'mean_g', 'mean_r', 'mean_i', 'phot_g_mean_mag', 'amp_1_r',
       'phi_1_r', 'amp_1_g', 'phi_1_g', 'amp_1_i', 'phi_1_i', 'amp_2_r',
       'phi_2_r', 'amp_2_g', 'phi_2_g', 'amp_2_i', 'phi_2_i', 'amp_3_r',
       'phi_3_r', 'amp_3_g', 'phi_3_g', 'amp_3_i', 'phi_3_i', 'ngooddet_g',
       'ngooddet_r', 'ngooddet_i'],
      dtype='object')

In [ ]:
tbl

The catalog provides a probability that the stars they have selected are actually RR Lyrae.  Define a selector to only choose *high-confidence* RR Lyrae:

In [40]:
wgood = tbl['prob_rrl'] > 0.95

### Exercise 2 Tasks:

    
* Compute the absolute r-band magnitude for the high-confidence RR Lyrae using the provided values of distance (in pc) and dust extinction E(B-V) (the `ebv` column).  
    * Use A_r = 2.27, so the extinction correction is $-2.27 \times E(B-V)$ in magnitudes
* Scatter plot the absolute magnitude versus catalog period (`best_period`).  Overplot the period-luminosity relation: $M_r = -1.6 \:\log_{10}(\frac{P}{0.6}) + 0.51$, where $P$ is the period in days.
    * HINT: There are some bad values, be sure to "zoom in" to see the majority of points
* That period-luminosity relationship is not a great fit--the distances in the table were derived from averages over all three ZTF bands.  Fit a new linear period-luminosity relationship to the high-confidence RR Lyrae ([`astropy.modeling`](https://docs.astropy.org/en/stable/modeling/index.html) or [`np.polyfit`](https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html) may be helpful) and overplot it.  
    * HINT: Be sure that the `x` coordinate you fit is the log of the period: $\log_{10}(\frac{P}{0.6})$.

## 3. Estimate distance from the Period-Luminosity relation.

Now let's compute the distance our period-luminosity relationship implies for the RR Lyrae in exercise 1.

We will need the E(B-V) value at the source location, which we are providing you:

**E(B-V) = 0.012 mag**

In [5]:
ztf_src_ebv = 0.012

But you *could* programatically get it like this:

In [4]:
# from astroquery.ipac.irsa.irsa_dust import IrsaDust
# from astropy.coordinates import SkyCoord

# ztf_src_sc = SkyCoord(mean_ra, mean_dec, unit='degree')

# ztf_src_ebv = IrsaDust.get_query_table(ztf_src_sc,section='ebv')['ext SFD mean'].value[0]
# ztf_src_ebv

For this exercise:

* Use the period-luminosity relationship and the best-fit period of the source from Exercise 1 to **estimate the  distance in parsecs to our RR Lyr star.**